<a href="https://colab.research.google.com/github/InhuKim/AI_proj_Team2/blob/main/meta_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import random
import os
import math

from glob import glob
import pandas as pd
import cv2
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

from torch.autograd import Variable

from torch.utils.data import TensorDataset
from sklearn.preprocessing import MinMaxScaler
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

from torch.utils.data import TensorDataset

import torchvision.models as models
from torchvision import transforms

# 데이터 처리

In [ ]:
CFG = {
    'IMG_SIZE':256,
    'LEARNING_RATE':1e-3,
    'SEED':41
}

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

#### 메타데이터

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {device} device")

Using cuda device


In [ ]:
time_series = pd.DataFrame()
target_label = pd.DataFrame()

In [ ]:
# 학습 데이터

path = '/content/drive/MyDrive/Colab_Notebooks/data/meta_data/train'
path2 = '/content/drive/MyDrive/Colab_Notebooks/data/train'
count = 0
row = pd.DataFrame()
train_file_name = []

all_img_path = []
label_list = []

for i in range(75):
    if i + 1 < 10:
        i_str = '0' + str(i +1)
    elif i + 1 > 10:
        i_str = str(i +1)
    elif i + 1 == 10:
        i_str = str(i + 1)

    file_list = os.listdir(path + '/CASE' + i_str)
    meta_list_py = [file for file in file_list if file.endswith('.csv')]
    train_file_name.extend(meta_list_py)

    img_list = os.path.join(path2, '/CASE{}'.format(i_str))


    for img in train_file_name:
            # get image path
            if os.path.isdir(path2 + img_list):
                img = img.replace("N_", "")
                img = img.replace(".csv", "")
                all_img_path.extend(glob(path2 + os.path.join(path2, img_list, 'image', '{}.jpg'.format(img))))
                all_img_path.extend(glob(path2 + os.path.join(path2, img_list, 'image', '{}.png'.format(img))))

    for l in meta_list_py:
        data = pd.read_csv(path + '/CASE' + i_str + '/' + l)
        time_series = pd.concat([time_series, data])

In [ ]:
# 정답 데이터

path = '/content/drive/MyDrive/Colab_Notebooks/data/meta_data/target'
count2 = 0
label_file_name = []

for j in range(75):
    if j + 1 < 10:
        j_str = '0' + str(j +1)
    elif j + 1 > 10:
        j_str = str(j +1)
    elif j + 1 == 10:
        j_str = str(j + 1)

    file_list = os.listdir(path + '/CASE' + j_str)
    label_list_py = [file for file in file_list if file.endswith('.csv')]
    label_file_name.append(label_list_py)

    for k in label_list_py:
        target = pd.read_csv(path + '/CASE' + j_str + '/' + k)
        target_label = pd.concat([target_label, target])

### 데이터셋 변환

In [ ]:
time_dataset = time_series.to_numpy()
time_dataset = time_dataset[:, 1:]

target_dataset = target_label.to_numpy()
target_dataset = target_dataset[:, 1]

In [ ]:
print(len(time_dataset) / 1440)

1292.0


In [ ]:
# def sliding_hours(data, target):
#     x = []
#     y = []

#     for i in range(len(target)):
#         x2 = []

#         for j in range(24):
#             _x = data[i][(j * 60): ((j+1) * 60)]
#             array = []
#             for k in range(18):
#                 _x1 = np.mean(_x[:, k])
#                 array.append(_x1)
#             x2.append(np.array(array))

#         x.append(np.array(x2))

#     return np.array(x), target

In [ ]:
# 하루 단위로 데이터를 분리하여 label과 맵핑

def sliding_windows(data, target):
    x = []
    y = []

    for i in range(len(target)):
        _x = data[(i * 1440): ((i + 1) * 1440)]
        _y = target[i]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)

# MinMaxScaler
sc_df = MinMaxScaler()
training_data = sc_df.fit_transform(time_dataset)

# 배치 사이즈
x, y = sliding_windows(training_data, target_dataset)

# x1, y1 = sliding_hours(x, y)


In [ ]:
train_size = int(len(y) * 0.7)
test_size = len(y) - train_size

trainX = Variable(torch.Tensor(np.array(x[:1020]))).to(device)
trainY = Variable(torch.Tensor(np.array(y[:1020]))).to(device)

testX = Variable(torch.Tensor(np.array(x[1020:1275]))).to(device)
testY = Variable(torch.Tensor(np.array(y[1020:1275]))).to(device)

In [ ]:
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

train_meta_dataset = TensorDataset(trainX, trainY)
valid_meta_dataset = TensorDataset(testX, testY)

torch.Size([1020, 1440, 18])
torch.Size([1020])
torch.Size([255, 1440, 18])
torch.Size([255])


In [ ]:
train_meta_dataloader = DataLoader(train_meta_dataset, batch_size=20, shuffle=True)
valid_meta_dataloader = DataLoader(valid_meta_dataset, batch_size=5, shuffle=False)

In [ ]:
for x, y in train_meta_dataloader:
    print(x.size(1))
    print(x[:,:,2])
    print(y[0])
    break

1440
tensor([[0.2921, 0.2921, 0.2921,  ..., 0.2599, 0.2599, 0.2599],
        [0.4706, 0.4716, 0.4716,  ..., 0.3943, 0.4025, 0.4128],
        [0.5088, 0.4964, 0.4665,  ..., 0.4314, 0.4675, 0.4974],
        ...,
        [0.7130, 0.7130, 0.7130,  ..., 0.7295, 0.7306, 0.7295],
        [0.7068, 0.7079, 0.7089,  ..., 0.5624, 0.6006, 0.6150],
        [0.4469, 0.4685, 0.4892,  ..., 0.4809, 0.4830, 0.4819]],
       device='cuda:0')
tensor(3.1580, device='cuda:0')


### 이미지

In [ ]:
def get_test_data(data_dir):
    # get image path
    img_path_list = glob(os.path.join(data_dir, 'image', '*.jpg'))
    img_path_list.extend(glob(os.path.join(data_dir, 'image', '*.png')))
    img_path_list.sort(key=lambda x:int(x.split('/')[-1].split('.')[0]))
    return img_path_list

In [ ]:
test_img_path = get_test_data('/content/drive/MyDrive/Colab_Notebooks/data/test')

In [ ]:
# Train : Validation = 0.8 : 0.2 Split
# train_len = int(len(all_img_path)*0.8)

train_img_path = all_img_path[:1020]
train_label = target_dataset[:1020]

vali_img_path = all_img_path[1020:1275]
vali_label = target_dataset[1020:1275]

In [ ]:
len(train_label)

1020

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, train_mode=True, transforms=None):
        self.transforms = transforms
        self.train_mode = train_mode
        self.img_path_list = img_path_list
        self.label_list = label_list

    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        # Get image data
        image = cv2.imread(img_path)
        if self.transforms is not None:
            image = self.transforms(image)

        if self.train_mode:
            label = self.label_list[index]
            return image, label
        else:
            return image
    
    def __len__(self):
        return len(self.img_path_list)

In [ ]:
train_transform = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Resize((CFG['IMG_SIZE'], CFG['IMG_SIZE'])),
                    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
                    ])

test_transform = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Resize((CFG['IMG_SIZE'], CFG['IMG_SIZE'])),
                    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
                    ])

In [ ]:
train_dataset = CustomDataset(train_img_path, train_label, train_mode=True, transforms=train_transform)
train_loader = DataLoader(train_dataset, batch_size = 20, shuffle=True, num_workers=0)

vali_dataset = CustomDataset(vali_img_path, vali_label, train_mode=True, transforms=test_transform)
vali_loader = DataLoader(vali_dataset, batch_size = 5, shuffle=False, num_workers=0)

In [ ]:
for img, label in iter(train_loader):
    print(img.shape)
    print(label)
    break

torch.Size([20, 3, 256, 256])
tensor([7.3890e+01, 4.5779e+01, 2.1610e+00, 3.0777e+02, 2.8610e+00, 6.2230e+00,
        9.4000e-02, 6.6800e-01, 3.1190e+00, 5.5091e+01, 3.3225e+01, 3.5720e+01,
        2.7206e+02, 3.4840e+01, 2.5334e+01, 2.7080e+00, 1.2129e+01, 6.2700e-01,
        1.5682e+02, 3.5283e+01], dtype=torch.float64)


# 모델

### Image Feature Extractor

In [ ]:
class CNNFeatureExtracter(torch.nn.Module):
    def __init__(self):
        super(CNNFeatureExtracter, self).__init__()
        model = models.resnet50(pretrained=True)
        modules = list(model.children())[:-1]
        self.feature_extract = nn.Sequential(*modules)
        self.regressor = nn.Linear(256*6*6, 1)
        

    def forward(self, x):
        x = self.feature_extract(x)
        # x = x.mean(dim=(-2, -1))
        # (batch, 2048, 4, 4)
        out = torch.flatten(x, start_dim=1)
        # out = self.regressor(x)
        return out

### RNN

In [ ]:
class RNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_layers, bidirectional):
        super(RNN, self).__init__()
        
        self.RNN = nn.RNN(input_size = input_size, hidden_size=hidden_size, num_layers=num_layers, bidirectional=bidirectional)
        self.fc = nn.Linear(hidden_size *2, 1)
        
        
    def forward(self, x):
        x, _ = self.RNN(x)
        out = self.fc(x)
        
        return out

### LSTM

In [ ]:
class LSTM(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, bidirectional):
        super(LSTM, self).__init__()
        
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.bidirectional = bidirectional
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True, bidirectional=bidirectional).to(device)
        
        self.fc = nn.Linear(hidden_size, 1).to(device)
        # self.fc2 = nn.Linear(256, 1).to(device)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size)).to(device)
        
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size)).to(device)
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        print(x.shape)
        print(h_0.shape)
        print(c_0.shape)
        print(h_out.shape)
        print(ula.shape)
                
        out = self.fc(h_out.view(-1, self.hidden_size))
        # out = self.fc2(out)
        
        return out

In [ ]:
model = LSTM(18, 512, 1, False)

In [ ]:
data = torch.tensor(np.arange(8*1440*18).reshape(8, 1440, 18), dtype=torch.float32).to(device)

In [ ]:
out = model(data)
print(out.shape)

torch.Size([8, 1440, 18])
torch.Size([1, 8, 512])
torch.Size([1, 8, 512])
torch.Size([1, 8, 512])
torch.Size([8, 1440, 512])
torch.Size([8, 1])


### Conv1d -time series

In [ ]:
from torch.nn.modules.pooling import MaxPool1d

class MultiConv(nn.Module):
    def __init__(self, n_features):
        super(MultiConv, self).__init__()
        self.n_features = n_features
        # self.n_hidden = n_hidden
        # self.n_layers = n_layers
        
        self.conv1d_layer1 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=60, stride=1).to(device)

        self.conv1d_layer2 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=60, stride=1).to(device)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool1d(kernel_size=2, stride=2)
        # self.lstm = nn.LSTM(input_size=n_features, hidden_size=n_hidden,
        #                     num_layers=n_layers, batch_first=True).to(device)

        # RNN으로 하는 경우 아래 레이어 추가
        # self.linear = nn.Linear(in_features=12420, out_features=2048).to(device)


    def forward(self, x):

        # lstm hidden state
        # h_0 = Variable(torch.zeros(
        #     self.n_layers, x.size(0), self.n_hidden)).to(device)
        
        # c_0 = Variable(torch.zeros(
        #     self.n_layers, x.size(0), self.n_hidden)).to(device)

        # multi head cnn
        for idx in range(self.n_features):
            _x = x[:,:, idx].unsqueeze(1)
            _x = self.conv1d_layer1(_x)
            _x = self.relu(_x).squeeze()
            
            if idx == 0:
                # _x = _x.unsqueeze(1)
                # conv_x = self.conv1d_layer2(_x).unsqueeze(1)
                # conv_x = self.relu(conv_x).squeeze()
                # conv_x = self.maxpool(conv_x).unsqueeze(2)
                conv_x = self.maxpool(_x).unsqueeze(2)

            elif idx > 0:
                # _x = _x.unsqueeze(1)
                # _x = self.conv1d_layer2(_x).unsqueeze(1)
                # _x = self.relu(_x).squeeze()
                # _x = self.maxpool(_x).unsqueeze(2)
                _x = self.maxpool(_x).unsqueeze(2)
                conv_x = torch.cat((conv_x, _x), dim=2)

        # lstm_out, (h_out, _) = self.lstm(conv_x, (h_0, c_0))
        # y_pred = self.linear(h_out.view(-1, self.n_hidden))

        out = torch.flatten(conv_x, start_dim=1)
        # out = self.linear(out)
        return out

### GRU

In [ ]:
# GRU

class GRUNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, drop_prob=0.2):
        super(GRUNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = 2
        
        self.gru = nn.GRU(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, 1)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):
        out, h = self.gru(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device)
        return hidden

In [ ]:
model = CNNFeatureExtracter()
model2 = MultiConv(18)

for img, label in train_loader:
    out = model(img)
    print(out.shape)
    break

for meta, label in train_meta_dataloader:
    out2 = model2(meta)
    print(out2.shape)
    break

torch.Size([16, 2048])
torch.Size([20, 12420])


### Mulit Input Model

In [ ]:
class MultiInput(nn.Module):
    def __init__(self, n_features):
        super(MultiInput, self).__init__()
        self.resnet = CNNFeatureExtracter()
        self.multiconv = MultiConv(n_features)
        # self.rnn = RNN(input_size, hidden_size, num_layers, bidirectional)
        self.linear = nn.Linear(2048 +12420, 1)
        
    def forward(self, img, meta):
        img_features = self.resnet(img)
        meta_features = self.multiconv(meta)
        total_features = torch.cat((img_features, meta_features), dim=1)

        out = self.linear(total_features)
        
        return out

# 학습

In [ ]:
# validation 함수

def validation(model, vali_loader, criterion, device):
    model.eval()
    vali_loss = []

    with torch.no_grad():
        for idx, sample in enumerate(vali_loader):
            x_vali, y_vali = sample

            out = model(x_vali)
            loss = criterion(out, y_vali)
            vali_loss.append(loss.item())

    vali_mae_loss = np.mean(vali_loss)
    return vali_mae_loss

In [ ]:
# validation 함수

def validation2(model, vali_loader, criterion, device):
    model.eval()
    vali_loss = []

    with torch.no_grad():
        for idx, sample in enumerate(vali_loader):
            x_vali, y_vali = sample

            h = model.init_hidden(5)
            h = h.data

            out, h = model(x_vali.to(device).float(), h)
            loss = criterion(out, y_vali)
            vali_loss.append(loss.item())

    vali_mae_loss = np.mean(vali_loss)
    return vali_mae_loss

In [ ]:
def validation3(model, vali_loader, valid_meta_dataloader, criterion, device):
    model.eval()
    vali_loss = []

    with torch.no_grad():
        for img_data, meta_data in tqdm(iter(zip(vali_loader, valid_meta_dataloader))):

            x_valid, y_valid = img_data
            x_meta_valid, meta_label = meta_data

            out = model(x_valid.to(device), x_meta_valid.to(device))
            loss = criterion(out, y_valid.to(device))
            vali_loss.append(loss.item())

    vali_mae_loss = np.mean(vali_loss)
    return vali_mae_loss

In [ ]:
# LSTM 학습

num_epochs = 50
learning_rate = 1e-3
optimizer_list = ['adam', 'rmsprop', 'sgd']

input_size = 18
hidden_size = 512
num_layers = 1

best_mae = 9999

# model = LSTM(input_size, hidden_size, num_layers, bidirectional=False).to(device)
# model = MultiConv(n_features=18, n_hidden=512, n_layers=1).to(device)

model = MultiInput(18).to(device)

criterion = torch.nn.L1Loss().to(device)    # mean-squared error for regression
#optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)


1
# Train the model
for opti in optimizer_list:
    if opti == 'sgd':
        optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
    elif opti == 'adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    elif opti == 'rmsprop':
        optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
    
    print("------------------------------------")
    print('optimizer - {}'.format(opti))
    for epoch in range(num_epochs):
        model.train()
        train_loss = []
        for img_data, meta_data in tqdm(iter(zip(train_loader, train_meta_dataloader))):
            
            x_train, label = img_data
            x_meta_train, meta_label = meta_data

            optimizer.zero_grad()
            outputs = model(x_train.to(device), x_meta_train.to(device))
            out = torch.squeeze(outputs)
        
            # obtain the loss function
            loss = criterion(out, label.to(device))
        
            loss.backward()
        
            optimizer.step()

            train_loss.append(loss.item())

            # validation
        vali_mae = validation3(model, vali_loader, valid_meta_dataloader, criterion, device)

        print(f'Epoch [{epoch}] Train MAE : [{np.mean(train_loss):.5f}] Validation MAE : [{vali_mae:.5f}]\n')

        if best_mae > vali_mae:
            best_mae = vali_mae
            if opti == 'sgd':
                torch.save(model.state_dict(), '/content/drive/MyDrive/Colab_Notebooks/competition/multi_resnet2cnnlinear_sgd.pth')
            elif opti == 'adam':
                torch.save(model.state_dict(), '/content/drive/MyDrive/Colab_Notebooks/competition/multi_resnet2cnnlinear_adam.pth')
            elif opti == 'rmsprop':
                torch.save(model.state_dict(), '/content/drive/MyDrive/Colab_Notebooks/competition/multi_resnet2cnnlinear_rmsprop.pth')
            print('------  Model Saved.  ---------')

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

------------------------------------
optimizer - adam


0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

Epoch [20] Train MAE : [56.44001] Validation MAE : [82.20214]



0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [ ]:
def train(train_loader, learn_rate, hidden_dim=512, EPOCHS=20, model_type="GRU"):
    
    # Setting common hyperparameters

    best_mae = 999
    

    # Instantiating the models
    if model_type == "GRU":
        model = GRUNet(input_dim, hidden_dim)
    elif model_type == "LSTM":
        model = LSTM(input_size, hidden_dim, num_layers, bidirectional)
    model.to(device)
    
    # Defining loss function and optimizer
    criterion = nn.L1Loss()
    optimizer = torch.optim.RMSprop(model.parameters(), lr=learn_rate)
    
    
    print("Starting Training of {} model".format(model_type))
    epoch_times = []

    # Start training loop
    for epoch in range(1,EPOCHS+1):
        model.train()
        train_loss = []
        h = model.init_hidden(20)
        avg_loss = 0.
        counter = 0
        for x, label in tqdm(iter(train_loader)):
            counter += 1
            if model_type == "GRU":
                h = h.data
            else:
                h = tuple([e.data for e in h])

            model.zero_grad()
            
            out, h = model(x.to(device).float(), h)
            loss = criterion(out, label.to(device).float())

            loss.backward()

            optimizer.step()

            train_loss.append(loss.item())

        vali_mae = validation2(model, valid_meta_dataloader, criterion, device)

        print(f'Epoch [{epoch}] Train MAE : [{np.mean(train_loss):.5f}] Validation MAE : [{vali_mae:.5f}]\n')

        if best_mae > vali_mae:
            best_mae = vali_mae
            torch.save(model.state_dict(), '/content/drive/MyDrive/Colab_Notebooks/competition/meta_gru.pth')
            print('model saved!')


# def evaluate(model, test_x, test_y, label_scalers):
#     model.eval()
#     outputs = []
#     targets = []
#     start_time = time.clock()
#     for i in test_x.keys():
#         inp = torch.from_numpy(np.array(test_x[i]))
#         labs = torch.from_numpy(np.array(test_y[i]))
#         h = model.init_hidden(inp.shape[0])
#         out, h = model(inp.to(device).float(), h)
#         outputs.append(label_scalers[i].inverse_transform(out.cpu().detach().numpy()).reshape(-1))
#         targets.append(label_scalers[i].inverse_transform(labs.numpy()).reshape(-1))
#     print("Evaluation Time: {}".format(str(time.clock()-start_time)))
#     sMAPE = 0
#     for i in range(len(outputs)):
#         sMAPE += np.mean(abs(outputs[i]-targets[i])/(targets[i]+outputs[i])/2)/len(outputs)
#     print("sMAPE: {}%".format(sMAPE*100))
#     return outputs, targets, sMAPE

# Run

In [ ]:
input_dim = next(iter(train_meta_dataloader))[0].shape[2]
output_dim = 1
n_layers = 2

model = GRUNet(input_dim, hidden_dim=256).to(device)

# optimizer = torch.optim.SGD(params = model.parameters(), lr = 0.003])
scheduler = None

train(train_meta_dataloader, learn_rate=0.03, hidden_dim=512, EPOCHS=20, model_type="GRU")

Starting Training of GRU model


  0%|          | 0/51 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([20])) that is different to the input size (torch.Size([20, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Epoch [1] Train MAE : [70.72741] Validation MAE : [76.08319]

model saved!


  0%|          | 0/51 [00:00<?, ?it/s]

...

Epoch [18] Train MAE : [69.99401] Validation MAE : [74.47565]



  0%|          | 0/51 [00:00<?, ?it/s]

Epoch [19] Train MAE : [70.01872] Validation MAE : [74.74328]



  0%|          | 0/51 [00:00<?, ?it/s]

Epoch [20] Train MAE : [69.83346] Validation MAE : [74.40825]

model saved!
